# Homework 3: N-Gram Language Models

## Total Points: 95 + 5 bonus
- **Overview**: In the textbook, language modeling was defined as the task of predicting the next word in a sequence given the previous words. In this assignment, we will focus on the related problem of predicting the next character in a sequence given the previous characters.

- **Learning goals**:
    - Understand how to compute language model probabilities using maximum likelihood estimation.
    - Implement basic smoothing, back-off and interpolation.
    - Have fun using a language model to probabilistically generate texts.
    - Use a set of language models to perform text classification.

- **Data**: We will provide you with training and development data that has been manually labeled. We will also give you a test set without labels. You will build a classifier to predict the labels on our test set. You can upload your classifier’s predictions to Gradescope. We will score its predictions and maintain a leaderboard showing whose classifier has the best performance.

- **Delieverables:** This assignment has several deliverables:
    - Your implementations for the functions in the skeleton code (this notebook)
    - Answers to all questions labeled as `Answer #.#` in a file named `report.pdf`
    - Your model’s output for the test set (your model will be ranked on a leaderboard against the other students’ outputs)

- **Grading**: We will use the auto-grading system called `PennGrader`. To complete the homework assignment, you should implement anything marked with `#TODO` and run the cell with `#PennGrader` note. **There will be no hidden tests in this assignment.** In other words, you will know your score once you finish all the `#TODO` and run all the `#PennGrader` tests!


## Recommended Readings
- [Language Modeling with N-grams](https://web.stanford.edu/~jurafsky/slp3/3.pdf). Dan Jurafsky and James H. Martin. Speech and Language Processing (3rd edition draft) .
- [A Bit of Progress in Language Modeling](https://arxiv.org/abs/cs/0108005). Joshua Goodman. Computer Speech and Language .
- [The Unreasonable Effectiveness of Character-level Language Models](http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139). Yoav Goldberg. Response to Andrej Karpathy's blog post. 2015.
- [Language Independent Authorship Attribution using Character Level Language Models](http://www.aclweb.org/anthology/E/E03/E03-1053.pdf). Fuchun Pen, Dale Schuurmans, Vlado Keselj, Shaojun Wan. EACL 2003.

## To get started, **make a copy** of this colab notebook into your google drive!

## Setup 1: PennGrader Setup [4 points]

In [1]:
## DO NOT CHANGE ANYTHING, JUST RUN
%%capture
!pip install penngrader-client

In [2]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Overwriting notebook-config.yaml


In [3]:
!cat notebook-config.yaml|

/bin/bash: -c: line 2: syntax error: unexpected end of file


In [4]:
from penngrader.grader import *

## TODO - Start
STUDENT_ID = 62502470 # YOUR PENN-ID GOES HERE AS AN INTEGER#
## TODO - End

SECRET = STUDENT_ID
grader = PennGrader('notebook-config.yaml', 'CIS5300_OL_23Su_HW3', STUDENT_ID, SECRET)

PennGrader initialized with Student ID: 62502470

Make sure this correct or we will not be able to store your grade


In [5]:
# check if the PennGrader is set up correctly
# do not chance this cell, see if you get 4/4!
name_str = 'Rui Jiang'
grader.grade(test_case_id = 'name_test', answer = name_str)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Setup 2: Dataset / Packages
- **Run the following cells without changing anything!**
- [Loading dataset from huggingface](https://huggingface.co/docs/datasets/v1.8.0/loading_datasets.html#from-local-files)

In [6]:
import math, random
from collections import Counter
from dill.source import getsource

In [7]:
## DO NOT CHANGE ANYTHING, JUST RUN

def get_class_source(cls):
    import re
    class_name = cls.__name__
    from IPython import get_ipython
    ipython = get_ipython()
    inputs = ipython.user_ns['In']
    pattern = re.compile(r'^\s*class\s+{}\b'.format(class_name))
    for cell in reversed(inputs):
        if pattern.search(cell):
            return cell
    return None

In [8]:
from pathlib import Path
import os
print('/content/cities_train.zip exist is {} '.format(os.path.exists('/content/cities_train.zip')))
print('/content/cities_val.zip exist is {} '.format(os.path.exists('/content/cities_val.zip')))
print('/content/cities_test.txt exist is {} '.format(os.path.exists('/content/cities_test.txt')))
print('/content/nytimes_article.txt exist is {} '.format(os.path.exists('/content/nytimes_article.txt')))
print('/content/shakespeare_sonnets.txt exist is {} '.format(os.path.exists('/content/shakespeare_sonnets.txt')))
print('/content/shakespeare_input.txt exist is {} '.format(os.path.exists('/content/shakespeare_input.txt')))


/content/cities_train.zip exist is True 
/content/cities_val.zip exist is True 
/content/cities_test.txt exist is True 
/content/nytimes_article.txt exist is True 
/content/shakespeare_sonnets.txt exist is True 
/content/shakespeare_input.txt exist is True 


In [9]:
!gdown 19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T # https://drive.google.com/file/d/19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T/view?usp=share_link
!unzip -o cities_train.zip
!gdown 1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu # https://drive.google.com/file/d/1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu/view?usp=sharing
!unzip -o cities_val.zip
!gdown 1mz6EdH-4pOwr-o3lHDlh1SS7oRdydNr1 # https://drive.google.com/file/d/1mz6EdH-4pOwr-o3lHDlh1SS7oRdydNr1/view?usp=sharing
!gdown 11es6r9fVz_wf_Q2T7oP2IQDkAAmhtzf4 # https://drive.google.com/file/d/11es6r9fVz_wf_Q2T7oP2IQDkAAmhtzf4/view?usp=share_link
!gdown 19P37gR_XDup-h3a4LdKtQWbdM-lGPqin # https://drive.google.com/file/d/19P37gR_XDup-h3a4LdKtQWbdM-lGPqin/view?usp=sharing
!gdown 1kKfh2oaxinBPVwWaux6FA9TerqAH5nHw # https://drive.google.com/file/d/1kKfh2oaxinBPVwWaux6FA9TerqAH5nHw/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T
To: /content/cities_train.zip
100% 160k/160k [00:00<00:00, 111MB/s]
Archive:  cities_train.zip
  inflating: train/af.txt            
  inflating: train/de.txt            
  inflating: train/fi.txt            
  inflating: train/fr.txt            
  inflating: train/in.txt            
  inflating: train/ir.txt            
  inflating: train/cn.txt            
  inflating: train/za.txt            
  inflating: train/pk.txt            
Downloading...
From: https://drive.google.com/uc?id=1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu
To: /content/cities_val.zip
100% 7.56k/7.56k [00:00<00:00, 25.4MB/s]
Archive:  cities_val.zip
  inflating: val/af.txt              
  inflating: val/cn.txt              
  inflating: val/de.txt              
  inflating: val/fi.txt              
  inflating: val/fr.txt              
  inflating: val/ir.txt              
  inflating: val/za.txt              
  inflating: val/pk.txt        

# Section 0: Generating N-Grams [5 points]
- **Problem 0:** Write a function `ngrams(n, text)` that produces a list of all n-grams of the specified size from the input text. Each n-gram should consist of a 2-element tuple `(context, char)`, where the context is itself an n-length string comprised of the n characters preceding the current character. The sentence should be padded with n ~ characters at the beginning (we’ve provided you with `start_pad(n)` for this purpose). If n=0 , all contexts should be empty strings. You may assume that n≥0.

```
>>> ngrams(1, 'abc')
[('~', 'a'), ('a', 'b'), ('b', 'c')]

>>> ngrams(2, 'abc')
[('~~', 'a'), ('~a', 'b'), ('ab', 'c')]
```

In [10]:
### DO NOT CHANGE ###
def start_pad(n):
    ''' Returns a padding string of length n to append to the front of text
        as a pre-processing step to building n-grams '''
    return '~' * n
### DO NOT CHANGE ###

In [11]:
def ngrams(n, text):
    ''' Returns the ngrams of the text as tuples where the first element is
        the length-n context and the second is the character '''

    ## AUTOGRADER HELPER ##
    def start_pad(n):
        ''' Returns a padding string of length n to append to the front of text
            as a pre-processing step to building n-grams '''
        return '~' * n
    ## AUTOGRADER HELPER ##


    ## TODO: create n-grams given the text
    padding = start_pad(n)
    new_text = padding+text
    text_len = len(text)

    padded_text = start_pad(n) + text
    result = []
    for i in range(text_len):
      result.append((new_text[i:n+i], new_text[n+i]))
    return result


In [ ]:
result = ngrams(2, "ab")
result

[('~~', 'a'), ('~a', 'b')]

In [ ]:
result = ngrams(0, "ab")
result

[('', 'a'), ('', 'b')]

In [ ]:
result = ngrams(1, "ab")
result

[('~', 'a'), ('a', 'b')]

In [12]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q0_ngrams', answer = getsource(ngrams))

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 1: Creating an N-Gram Model [36 points]
In this section, you will build a simple n-gram language model that can be used to generate random text resembling a source document. Your use of external code should be limited to built-in Python modules, which excludes, for example, `NumPy` and `NLTK`.


In [13]:
from collections import defaultdict
import math
COUNT_KEY = 'count'
CHARS_KEY = 'chars'


In [14]:
class NgramModel(object):
    ''' A basic n-gram model using add-k smoothing '''

    def __init__(self, n, k = 0):
        # print("__init__")
        # About `k`: you don't need to worry about it for problem 1.1, it is something you will need in problem 2.1
        self.k = k

        # contexts is a dictionary with the context as keys and corresponding value is a dictionary of count and
        # chars which is a dictionary of characters as key and it's count as value.
        # Please do not change the format of storing the counts and characters, the autograder uses this structure
        # For example:
        # if the context is 'ab' with count=3 and characters following it are 'a' and 'b' with counts 2 and 1 respectively
        # then the context dictionary would look like this:
        # {
        #   'ab' : {'count': 3,'chars': {'a' : 2, 'b' : 1}},
        #   'aa' : {...},
        #   ...
        # }
        self.contexts = {}

        ## TODO 1.1 - see problem 1.1
        self.n = n
        self.vocab = set()

    def get_vocab(self):
        # print("get_vocab")
        ''' Returns the set of characters in the vocab '''
        return self.vocab

    def get_context(self):
        # print("get_context")
        ''' Returns the dictionary for context '''
        return self.contexts

    def update(self, text):
        ''' Updates the model n-grams based on text '''
        ## TODO 1.2 - see problem 1.2
        # print("get_update")
        if self.n == 0:
            if '' in self.contexts:
                self.contexts[''][COUNT_KEY] = self.contexts[''][COUNT_KEY] + len(text)
            else:
                self.contexts[''] = dict()
                self.contexts[''][COUNT_KEY] = len(text)
                self.contexts[''][CHARS_KEY] = defaultdict(int)
            for letter in text:
                self.contexts[''][CHARS_KEY][letter] = self.contexts[''][CHARS_KEY][letter] + 1
        else:
            ngrams_context = ngrams(self.n, text)
            # print("ngrams_context is {}".format(ngrams_context))
            for ngram_item in ngrams_context:
                pre_context, cur_letter = ngram_item
                if pre_context in self.contexts:
                    self.contexts[pre_context][COUNT_KEY] += 1
                else:
                    self.contexts[pre_context] = dict()
                    self.contexts[pre_context][COUNT_KEY] = 1
                    self.contexts[pre_context][CHARS_KEY] = defaultdict(int)
                self.contexts[pre_context][CHARS_KEY][cur_letter] += 1
        # update vocalbulary
        for key, value in self.contexts.items():
            # self.vocabulary.update(key)
            self.vocab.update(value[CHARS_KEY].keys())

        # print("get_vocab self.vocab is {}".format(self.vocab))
        if '~' in self.vocab:
            self.vocab.remove('~')
        return

    def prob(self, context, char):
        # print("prob")
        ''' Returns the probability of char appearing after context '''
        if self.k == 0:
            ## TODO 1.2 - see problem 1.2
            # print("self.prob self.contexts {} context is {} char is {}".format(self.contexts, context, char))

            if context not in self.contexts:
                # print("context is not in sefl.contexts ")
                # print("self.vocabulary is {}".format(self.vocabulary))
                return 1/(len(self.get_vocab()))
            context_count = self.contexts[context][COUNT_KEY]
            char_count = self.contexts[context][CHARS_KEY][char]
            return char_count / context_count
        else:
            ## TODO 2.1 - see problem 2.1 Smoothing
            if context not in self.contexts:
                return 1/(len(self.get_vocab()))
            v_count = len(self.get_vocab())
            sum_n = 0
            char_count = self.k
            for key in self.contexts[context][CHARS_KEY]:
                if key == char:
                    char_count += self.contexts[context][CHARS_KEY][key]
                sum_n += self.contexts[context][CHARS_KEY][key]
            return char_count / (sum_n + self.k * v_count)

    def random_char(self, context):
        # print("random_char")
        ''' Returns a random character based on the given context and the
            n-grams learned by this model '''
        ## TODO 1.3 - see problem 1.3
        if context == '':
            # select one letter radomly from vacabulary
            r = random.random()
            voc = sorted(list(self.get_vocab()))
            return voc[int(r * len(voc))]

        if context not in self.contexts:
            voc = sorted(list(self.get_vocab()))
            r = random.random()
            # print("voc is {} r is {}".format(voc, r))
            return voc[int(r * len(voc))]

        total_count = self.contexts[context][COUNT_KEY]
        keys_list = sorted(list(self.contexts[context][CHARS_KEY].keys()))
        prob_range = {} # key is letters, value is tuple (lower range, uppper range) for probabilities
        prob = 0.0
        for key in keys_list:
            key_prob = self.contexts[context][CHARS_KEY][key] / total_count
            prob_range[key] = (prob, prob + key_prob)
            prob += key_prob
        # print("prob_range is {}".format(prob_range))
        r = random.random()
        for key in keys_list:
            if r >= prob_range[key][0] and r < prob_range[key][1]:
                return key

    def random_text(self, length):
        # print("random_text")
        ''' Returns text of the specified character length based on the
            n-grams learned by this model '''
        ## TODO 1.4 - see problem 1.4
        result = []
        context = ''

        for i in range(length):
            context = ''
            if len(result) < (self.n):
                context = '~' * (self.n-len(result)) + ''.join(result)
            else:
                # print("context is {}".format(result[(0-self.n):]))
                context = ''.join(result[(0-self.n):])
            # print("context is {}".format(context))
            result.append(self.random_char(context))
        return ''.join(result)

    def perplexity(self, text):
        # print("perplexity")
        ''' Returns the perplexity of text based on the n-grams learned by
            this model '''
        ## TODO 2.2 - see problem 2.2
        # return 0.56
        """
        >>> ngrams(1, 'abc')
        [('~', 'a'), ('a', 'b'), ('b', 'c')]

        >>> ngrams(2, 'abc')
        [('~~', 'a'), ('~a', 'b'), ('ab', 'c')]
        """
        grams = ngrams(self.n, text)
        # print("grams is {}".format(grams))
        sum = 0
        for gram in grams:
            pre_context = gram[0]
            cur_letter = gram[1]
            prob = self.prob(pre_context, cur_letter)
            if prob == 0.0:
                return math.inf
            # print("gram[0] is {} gram[1] is {} prob is {}".format(gram[0], gram[1], prob))
            log_new = math.log2(prob)
            # print("gram {} prob is {} log_new {}".format(gram, prob, log_new))
            sum += log_new

        avg = sum / len(grams)
        # print("avg is {} len of gram is {}".format(avg, len(grams)))
        return math.pow(2, 0 - avg)



## 1.1 Initialization [3 points]
In the `NgramModel` class, write an initialization method `__init__(self, n, k)` which stores the order n of the model and initializes any necessary internal variables. Then write a method `get_vocab(self)` that returns the vocab (this is the `set` of all characters used by this model).

- **Problem 1.1:** finish `__init__(self, n, k)` and `get_vocab(self)` [3 points]

In [15]:
# PennGrader - DO NOT CHANGE
ngram_model = NgramModel(1, 0)
grader.grade(test_case_id = 'test_q11_init_get_vocab', answer = get_class_source(NgramModel))

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.2 Update and Calculate Probabilities [15 points]
Write a method `update(self, text)` which computes the n-grams for the input sentence and updates the internal counts. Also write a method `prob(self, context, char)` which accepts an n-length string representing a context and a character, and returns the probability of that character occuring, given the preceding context. If you encounter a novel `context`, the probability of any given `char` should be 1/V where V is the size of the vocab.

- **Problem 1.2:** finish `update(self, text)` and `prob(self, context, char)` [15 points]

In [ ]:
n = 2
my_model = NgramModel(2)
for text in ['ababa', 'abcdddd', 'babcdabag']:
    my_model.update(text)

my_model.contexts

{'~~': {'count': 3, 'chars': defaultdict(int, {'a': 2, 'b': 1})},
 '~a': {'count': 2, 'chars': defaultdict(int, {'b': 2})},
 'ab': {'count': 5, 'chars': defaultdict(int, {'a': 3, 'c': 2})},
 'ba': {'count': 3, 'chars': defaultdict(int, {'b': 2, 'g': 1})},
 'bc': {'count': 2, 'chars': defaultdict(int, {'d': 2})},
 'cd': {'count': 2, 'chars': defaultdict(int, {'d': 1, 'a': 1})},
 'dd': {'count': 2, 'chars': defaultdict(int, {'d': 2})},
 '~b': {'count': 1, 'chars': defaultdict(int, {'a': 1})},
 'da': {'count': 1, 'chars': defaultdict(int, {'b': 1})}}

In [ ]:
# PennGrader - DO NOT CHANGE
all_test_cases = [
    (1, ['abab', 'abcd'], [('b','c'), ('a', '~')]),
    (2, ['ababa', 'abcdddd', 'babcdabag'], [('ab', 'a'), ('ad', 'b')]),
    (3, ['ababa', 'abcdddd', 'babcdabag'], [('ddd', 'd'   ), ('~~c', 'f')]),
    (2, ['abc'], [('~a', 'b'), ('~~', 'a')])
]

all_test_results = []
for test_data in all_test_cases:
    nm = NgramModel(test_data[0], 0)
    for text in test_data[1]:
        nm.update(text)
    vocab = nm.get_vocab()
    probs = []
    for prob_check in test_data[2]:
        prob = nm.prob(prob_check[0], prob_check[1])
        probs.append(prob)
    all_test_results.append((vocab, probs))

# reload_grader()
grader.grade(test_case_id = 'test_q12_update_prob', answer = all_test_results)

Correct! You earned 15/15 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.3 Random Character [6 points]

Write a method `random_char(self, context)` which returns a random character according to the probability distribution determined by the given context. Specifically, let $V=\langle v_1,v_2, \cdots, v_n \rangle$ be the vocab, sorted according to Python's natural lexicographic ordering, and let $r$ be a random number between 0 and 1. Your method should return the character $v_i$ such that:
$$
\sum_{j=1}^{i-1} P(v_j\ |\ \text{context}) \le r < \sum_{j=1}^i P(v_j\ | \ \text{context}).
$$
You should use a single call to the `random.random()` function to generate $r$, for example:
```
>>> m = NgramModel(0, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> random.seed(1)
>>> [m.random_char('') for i in range(25)]
['a', 'c', 'c', 'a', 'b', 'b', 'b', 'c', 'a', 'a', 'c', 'b', 'c', 'a', 'b', 'b', 'a', 'd', 'd', 'a', 'a', 'b', 'd', 'b', 'a']
```

- **Problem 1.3:** finish `random_char(self, context)` [6 points]

In [16]:
# PennGrader - DO NOT CHANGE
corpus = ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab']
nm = NgramModel(1, 0)
for text in corpus:
    nm.update(text)

random_seed_lst = [2, 3, 4]
random_char_lst = []
for seed in random_seed_lst:
    random.seed(seed)
    random_char = [nm.random_char('') for i in range(25)]
    random_char_lst.append(random_char)

grader.grade(test_case_id = 'test_q13_rand_char', answer = random_char_lst)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.4 Random Text [6 points]
In the `NgramModel` class, write a method `random_text(self, length)` which returns a string of characters chosen at random using the `random_char(self, context)` method. Your starting context should always be $n$ ~ characters, and the context should be updated as characters are generated. If $n=0$, your context should always be the empty string. You should continue generating characters until you've produced the specified number of random characters, then return the full string.

```python
>>> m = NgramModel(1, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> random.seed(1)
>>> m.random_text(25)
abcdbabcdabababcdddabcdba
```

- **Problem 1.4:** finish `random_text(self, length)` [6 points]

In [17]:
# PennGrader - DO NOT CHANGE
corpus = ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab']
nm = NgramModel(1, 0)
for text in corpus:
    nm.update(text)

random_seed_lst = [42, 43, 44]
random_text_lst = []
for seed in random_seed_lst:
    random.seed(seed)
    random_text = nm.random_text(25)
    random_text_lst.append(random_text)

grader.grade(test_case_id = 'test_q14_rand_text', answer = random_text_lst)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.5 Writing Shakespeare [6 points]

Now you can train a language model. We have provided this corpus of Shakespeare at `shakespeare_input.txt` (if you cannot find it, re-run the `Setup 2: Dataset / Packages` section).

We’ve also given you the function `create_ngram_model(model_class, path, n, k)` that will create and return an n-gram model trained on the entire file path provided and `create_ngram_model_lines(model_class, path, n, k)` that will create and return an n-gram model trained line-by-line on the file path provided. You should use the first for the Shakespeare file and the second for the city name files.

Try generating some Shakespeare with different order n-gram models. You should try running the following commands:

```python
>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 3)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 4)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 7)
>>> m.random_text(250)
```

What do you think? Is it as good as [1000 monkeys working at 1000 typewriters](https://www.youtube.com/watch?v=no_elVGGgW8)?

After generating a bunch of short passages, do you notice anything? *They all start with F!* In fact, after we hit a certain order, the first word is always *First*?  Why is that? Is the model trying to be clever? Is it saying *First, I will generate the word "First"*?  No, probably not.  Explain what is going on in your writeup.

- **Answer 1.5:** Generate Shakespear texts with at least 3 numbers of n and discuss on the results [6 points]
  - Write your response to this in your pdf

In [18]:
## DO NOT CHANGE ##
def create_ngram_model(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        model.update(f.read())
    return model

def create_ngram_model_lines(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        for line in f:
            model.update(line.strip())
    return model

In [ ]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2)
m.random_text(250)

"Fir;\nHESTERTITUS:\nI fol'end hume peas no ack:\nHOP Orlivaltimsen,\nI hor LET:\nAy, I com met my youggaishe fe.\nI asixessell cous, thipponswayse vaur bothon\ntat tooking to madefte, th to men,\nMy bow, to ther.\n\nSLEONESS Edweight tureciust to lat se bust o"

In [ ]:
m2 = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2)
len(m2.contexts.keys())

1639

In [ ]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 3)
m.random_text(250)

"First us way!\nThe proclaim all yourself.\nMonders two obsent, burn to low--a pice Poor rogue;' say deside hop\nIn theek 'tis by ther him?\n\nLORIZEL:\nO, if the slike and so passist:\n\nBERO:\nNay,\nOur be an is your his Ermet upon thout the rule breath willa"

In [ ]:
m3 = create_ngram_model(NgramModel, 'shakespeare_input.txt', 3)
len(m3.contexts.keys())

15723

In [ ]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 4)
m.random_text(250)

"First seems, some child holy officer.\nLavinia,\nDoing: you,\nmuch peace, get his chirping\nA mark'd,\nChamber\nThink my tears!\nI have of myself, true,\nAnd long.\n\nSecondemn'd, abour.\n\nYOUNG SIWARD IV:\nMy will give't herribly girl, and malice be wine moreov"

In [ ]:
m4 = create_ngram_model(NgramModel, 'shakespeare_input.txt', 4)
len(m4.contexts.keys())

82652

In [ ]:
model1 = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2, 1)
model1.contexts

{'~~': {'count': 1, 'chars': defaultdict(int, {'F': 1})},
 '~F': {'count': 1, 'chars': defaultdict(int, {'i': 1})},
 'Fi': {'count': 1137,
  'chars': defaultdict(int,
              {'r': 873,
               'v': 21,
               'e': 84,
               'l': 17,
               'x': 4,
               'f': 24,
               'n': 46,
               'g': 6,
               't': 17,
               'd': 11,
               's': 34})},
 'ir': {'count': 11139,
  'chars': defaultdict(int,
              {'s': 1564,
               ' ': 3213,
               't': 583,
               'e': 1337,
               ',': 1459,
               ':': 127,
               ';': 241,
               'd': 358,
               'c': 93,
               '\n': 127,
               'm': 154,
               'g': 87,
               '.': 366,
               'i': 448,
               'n': 14,
               '?': 162,
               "'": 15,
               'r': 225,
               '!': 92,
               'o': 101,
               

In [ ]:
model1 = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2, 1)
model1.contexts

{'~~': {'count': 1, 'chars': defaultdict(int, {'F': 1})},
 '~F': {'count': 1, 'chars': defaultdict(int, {'i': 1})},
 'Fi': {'count': 1137,
  'chars': defaultdict(int,
              {'r': 873,
               'v': 21,
               'e': 84,
               'l': 17,
               'x': 4,
               'f': 24,
               'n': 46,
               'g': 6,
               't': 17,
               'd': 11,
               's': 34})},
 'ir': {'count': 11139,
  'chars': defaultdict(int,
              {'s': 1564,
               ' ': 3213,
               't': 583,
               'e': 1337,
               ',': 1459,
               ':': 127,
               ';': 241,
               'd': 358,
               'c': 93,
               '\n': 127,
               'm': 154,
               'g': 87,
               '.': 366,
               'i': 448,
               'n': 14,
               '?': 162,
               "'": 15,
               'r': 225,
               '!': 92,
               'o': 101,
               

In [ ]:
modelline1_3 = create_ngram_model_lines(NgramModel, 'shakespeare_input.txt', 2, 2.0)


In [ ]:
modelline1_4 = modelline1_3
modelline1_4.perplexity("Against him first:")

7.563851325126464

In [ ]:
modelline1_5 = create_ngram_model_lines(NgramModel, 'shakespeare_input.txt', 2, 3.0)

In [ ]:
modelline1_5.perplexity("Against him first:")

7.6707916010000154

In [ ]:
model2 = create_ngram_model(NgramModel, 'shakespeare_input.txt', 3, 1)
text = "Note me this, good friend;"
perplexity = model1.perplexity(text)
perplexity

In [ ]:
## Answer 1.5 in your PDF

# Section 2: Smoothing, Perplexity, and Interpolation [30 points]

In this part of the assignment, you'll adapt your code in order to implement several of the  techniques described in [Section 3.5 of the Jurafsky and Martin textbook](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

## 2.1 Smoothing [6 points]

Laplace Smoothing is described in section 3.5.1 of the book. Laplace smoothing adds one to each count (hence its alternate name *add-one smoothing*). Since there are *V* characters in the vocabulary and each one was incremented, we also need to adjust the denominator to take into account the extra V observations.

$$P_{Laplace}(w_i) = \frac{count_i + 1}{N+|V|}$$

A variant of Laplace smoothing is called *Add-k smoothing* or *Add-epsilon smoothing*. This is described in section *Add-k 3.5.2*.

```python
>>> m = NgramModel(1, 1)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.prob('a', 'a')
0.14285714285714285
>>> m.prob('a', 'b')
0.5714285714285714
>>> m.prob('c', 'd')
0.4
>>> m.prob('d', 'a')
0.25
```

- **Problem 2.1**: Update your `NgramModel` code from Part 1 to implement add-k smoothing. (see comment of `## TODO 2.1`) [6 points]




In [ ]:
m = NgramModel(1, 1)
m.update('abab')
m.update('abcd')
m.prob('a', 'a')
m.prob('a', 'b')
m.prob('c', 'd')
m.prob('d', 'a')

0.25

In [ ]:
m.get_vocab()

{'a', 'b', 'c', 'd'}

In [19]:
# PennGrader - DO NOT CHANGE
smoothing_test_cases = [
    (1, 1, ['abab', 'abcd'], ('b', 'a')),
    (2, 1, ['ababa', 'abcdddd', 'babcdabag'], ('ba', 'b')),
    (3, 2, ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab'], ('aba', 'b'))
]
smoothing_test_results = []
for i, j, corpus, test_words in smoothing_test_cases:
    nm = NgramModel(i, j)
    for text in corpus:
        nm.update(text)
    smoothing_test_results.append(nm.prob(test_words[0], test_words[1]))

grader.grade(test_case_id = 'test_q21_smoothing', answer = smoothing_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [20]:
filename1 = "shakespeare_sonnets.txt"
filename2 = "nytimes_article.txt"

In [ ]:
model_2_2_n2_k1 = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2, 1)
with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = model_2_2_n2_k1.perplexity(text)
    print("filaname1 {} perplexity {}".format(filename1, perplexity))
with open(filename2, 'r', encoding='utf-8', errors="ignore") as f:
    text = f.read()
    perplexity = model_2_2_n2_k1.perplexity(text)
    print("filaname2 {} perplexity {}".format(filename2, perplexity))

filaname1 shakespeare_sonnets.txt perplexity 7.910100720888779
filaname2 nytimes_article.txt perplexity 11.018771417968004


In [ ]:
def readlines(path):
    lines = []
    with open(path, encoding='utf-8', errors='ignore') as f:
        for line in f:
          lines.append(line)
    return lines

model_2_2_n2_k1 = create_ngram_model(NgramModel, 'shakespeare_input.txt', 5, 2)
nytimes_article = readlines("nytimes_article.txt")
shakespeare_sonnets = readlines("shakespeare_sonnets.txt")
r1 = model_2_2_n2_k1.perplexity("".join(nytimes_article[:10]))
r2 = model_2_2_n2_k1.perplexity("".join(shakespeare_sonnets[:10]))
r3 = model_2_2_n2_k1.perplexity("a")
print("r1 {} r2 {} r3 {}".format(r1, r2, r3))

r1 17.97920259259353 r2 10.633397019486583 r3 67.5


In [ ]:
model_2_2_n3_k1 = create_ngram_model(NgramModel, 'shakespeare_input.txt', 3, 1)
with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = model_2_2_n3_k1.perplexity(text)
    print("filaname1 {} perplexity {}".format(filename1, perplexity))
with open(filename2, 'r', encoding='utf-8', errors="ignore") as f:
    text = f.read()
    perplexity = model_2_2_n3_k1.perplexity(text)
    print("filaname2 {} perplexity {}".format(filename2, perplexity))

filaname1 shakespeare_sonnets.txt perplexity 6.122231996035228
filaname2 nytimes_article.txt perplexity 9.78885334571599


In [ ]:
model_2_2_n4_k1 = create_ngram_model(NgramModel, 'shakespeare_input.txt', 4, 1)
with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = model_2_2_n4_k1.perplexity(text)
    print("filaname1 {} perplexity {}".format(filename1, perplexity))
with open(filename2, 'r', encoding='utf-8', errors="ignore") as f:
    text = f.read()
    perplexity = model_2_2_n4_k1.perplexity(text)
    print("filaname2 {} perplexity {}".format(filename2, perplexity))

filaname1 shakespeare_sonnets.txt perplexity 6.485626437020863
filaname2 nytimes_article.txt perplexity 10.851641044148732


## 2.2 Perplexity [12 points]

How do we know whether a language model is good? There are two basic approaches:
1. Task-based evaluation (also known as **extrinsic** evaluation), where we use the language model as part of some other task, like automatic speech recognition, or spelling correcktion, or an OCR system that tries to covert a professor's messy handwriting into text.
2. **Intrinsic** evaluation: Intrinsic evaluation tries to directly evalute the goodness of the language model by seeing how well the probability distributions that it estimates are able to explain some previously unseen test set.

Here's what the textbook says:

> For an intrinsic evaluation of a language model we need a test set. As with many of the statistical models in our field, the probabilities of an N-gram model come from the corpus it is trained on, the training set or training corpus. We can then measure the quality of an N-gram model by its performance on some unseen data called the test set or test corpus. We will also sometimes call test sets and other datasets that are not in our training sets held out corpora because we hold them out from the training data.

> So if we are given a corpus of text and want to compare two different N-gram models, we divide the data into training and test sets, train the parameters of both models on the training set, and then compare how well the two trained models fit the test set.

> But what does it mean to "fit the test set"? The answer is simple: whichever model assigns a higher probability to the test set is a better model.

We'll implement the most common method for intrinsic metric of language models: *perplexity*.  The perplexity of a language model on a test set is the inverse probability of the test set, normalized by the number of characters. For a test set $W = w_1 w_2 ... w_N$:

$$Perplexity(W) = P(w_1 w_2 ... w_N)^{-\frac{1}{N}}$$

$$ = \sqrt[N]{\frac{1}{P(w_1 w_2 ... w_N)}}$$

$$ = \sqrt[N]{\prod_{i=1}^{N}{\frac{1}{P(w_i \mid w_1 ... w_{i-1})}}}$$

Now implement the `perplexity(self, text)` function in `NgramModel`. A couple of things to keep in mind:
1. Numeric underflow is going to be a problem, so consider using logs.
2. Perplexity is undefined if the language model assigns any zero probabilities to the test set. In that case your code should return positive infinity - `float('inf')`.
3. On your unsmoothed models, you'll definitely get some zero probabilities for the test set. To test you code, you should try computing perplexity on the training set, and you should compute perplexity for your language models that use smoothing and interpolation.

```python
>>> m = NgramModel(1, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.perplexity('abcd')
1.189207115002721
>>> m.perplexity('abca')
inf
>>> m.perplexity('abcda')
1.515716566510398
```

In [ ]:
m = NgramModel(1, 0)
m.update('abab')
m.update('abcd')
m.perplexity('abcd')

1.189207115002721

In [ ]:
m.perplexity('abca')

inf

In [ ]:
model_2_2_n2_k1 = create_ngram_model(NgramModel, 'shakespeare_input.txt', n=2, k=1)


In [ ]:
m = NgramModel(2, 0)

for text in ['ababa', 'abcdddd', 'babcdabag']:
    m.update(text)

m.contexts

{'~~': {'count': 3, 'chars': defaultdict(int, {'a': 2, 'b': 1})},
 '~a': {'count': 2, 'chars': defaultdict(int, {'b': 2})},
 'ab': {'count': 5, 'chars': defaultdict(int, {'a': 3, 'c': 2})},
 'ba': {'count': 3, 'chars': defaultdict(int, {'b': 2, 'g': 1})},
 'bc': {'count': 2, 'chars': defaultdict(int, {'d': 2})},
 'cd': {'count': 2, 'chars': defaultdict(int, {'d': 1, 'a': 1})},
 'dd': {'count': 2, 'chars': defaultdict(int, {'d': 2})},
 '~b': {'count': 1, 'chars': defaultdict(int, {'a': 1})},
 'da': {'count': 1, 'chars': defaultdict(int, {'b': 1})}}

In [ ]:
result = m.perplexity('ba')
result


1.7320508075688774

In [ ]:
# PennGrader - DO NOT CHANGE rjiang
ppl_test_cases = [
    (1, 0, ['abab', 'abcd'], 'abca'),
    (2, 0, ['ababa', 'abcdddd', 'babcdabag'], 'ba'),
    (3, 1, ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab'], 'abcdddddddddd')
]
ppl_test_results = []
for i, j, corpus, test_words in ppl_test_cases:
    nm = NgramModel(i, j)
    for text in corpus:
        nm.update(text)
    ppl_test_results.append(nm.perplexity(test_words))

ppl_test_results

[inf, 1.7320508075688774, 2.9795392507641947]

- **Problem 2.2**: implement the `perplexity(self, text)` function in `NgramModel` [6 points]

In [21]:
# PennGrader - DO NOT CHANGE
ppl_test_cases = [
    (1, 0, ['abab', 'abcd'], 'abca'),
    (2, 0, ['ababa', 'abcdddd', 'babcdabag'], 'ba'),
    (3, 1, ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab'], 'abcdddddddddd')
]
ppl_test_results = []
for i, j, corpus, test_words in ppl_test_cases:
    nm = NgramModel(i, j)
    for text in corpus:
        nm.update(text)
    ppl_test_results.append(nm.perplexity(test_words))

grader.grade(test_case_id = 'test_q22_ppl', answer = ppl_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- **Answer 2.2:** Compare and discuss the perplexity for text that is similar and different from Shakespeare's plays. We provide you two dev text files, a New York Times article (`nytimes_article.txt`) and several of Shakespeare's sonnets (`shakespeare_sonnets.txt`); if you cannot find them, re-run `Setup 2: Dataset / Package` at the beginning. Also, feel free to experiment with your own text. [6 points]

In [ ]:
## Answer 2.2 in your PDF

## 2.3 Interpolation [12 points]

The idea of interpolation is to calculate the higher order n-gram probabilities also combining the probabilities for lower-order n-gram models. Like smoothing, this helps us avoid the problem of zeros if we haven't observed the longer sequence in our training data. Here's the math:

$$P_{interpolation}(w_i|w_{i−2} w_{i−1}) = \lambda_1 P(w_i|w_{i−2} w_{i−1}) + \lambda_2 P(w_i|w_{i−1}) + \lambda_3 P(w_i)$$

where $\lambda_1 + \lambda_2 + \lambda_3 = 1$.

We've provided you with another class definition `NgramModelWithInterpolation` that extends `NgramModel` for you to implement interpolation. If you've written your code robustly, you should only need to override the `get_vocab(self)`, `update(self, text)`, and `prob(self, context, char)` methods, along with the initializer.

The value of $n$ passed into `__init__(self, n, k)` is the highest order n-gram to be considered by the model (e.g. $n=2$ will consider 3 different length n-grams). Add-k smoothing should take place only when calculating the individual order n-gram probabilities, not when calculating the overall interpolation probability.

By default set the lambdas to be equal weights, but you should also write a helper function that can be called to overwrite this default. Setting the lambdas in the helper function can either be done heuristically or by using a development set, but in the example code below, we've used the default.

```python
>>> m = NgramModelWithInterpolation(1, 0)
>>> m.update('abab')
>>> m.prob('a', 'a')
0.25
>>> m.prob('a', 'b')
0.75

>>> m = NgramModelWithInterpolation(2, 1)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.prob('~a', 'b')
0.4682539682539682
>>> m.prob('ba', 'b')
0.4349206349206349
>>> m.prob('~c', 'd')
0.27222222222222225
>>> m.prob('bc', 'd')
0.3222222222222222
```
- **Problem 2.3**: implement the `NgramModelWithInterpolation` class [6 points]

In [ ]:
m = NgramModel(1, 0)
m.update('abab')
m.contexts

{'~': {'count': 1, 'chars': defaultdict(int, {'a': 1})},
 'a': {'count': 2, 'chars': defaultdict(int, {'b': 2})},
 'b': {'count': 1, 'chars': defaultdict(int, {'a': 1})}}

In [22]:
import math

class NgramModelWithInterpolation(NgramModel):
    ''' An n-gram model with interpolation '''

    def __init__(self, n, k):
        ## TODO 2.3
        self.n = n
        self.k = k
        self.num_model = n + 1

        self.models = [] # the first is unigram
        self.lambdas = [] # the length is either 1, 2, or 3. Maximum is 3.
        self.vocab = []
        for i in range(self.n+1):
            self.lambdas.append(1/(self.n+1))
            self.models.append(dict())
            self.vocab.append(set())

    def set_lambdas(self, lambdas):
        self.lambdas = lambdas

    def get_vocab(self):
        ## TODO 2.3
        return self.vocab

    def update(self, text):
        ## TODO 2.3
        for i in range(self.n+1):
            contexts = self.models[i]
            vocab = self.vocab[i]
            if i == 0:
                if '' in contexts:
                    contexts[''][COUNT_KEY] = contexts[''][COUNT_KEY] + len(text)
                else:
                    contexts[''] = dict()
                    contexts[''][COUNT_KEY] = len(text)
                    contexts[''][CHARS_KEY] = defaultdict(int)
                for letter in text:
                    contexts[''][CHARS_KEY][letter] = contexts[''][CHARS_KEY][letter] + 1
            else:
                ngrams_context = ngrams(i, text)
                for ngram_item in ngrams_context:
                    pre_context, cur_letter = ngram_item
                    if pre_context in contexts:
                        contexts[pre_context][COUNT_KEY] += 1
                    else:
                        contexts[pre_context] = dict()
                        contexts[pre_context][COUNT_KEY] = 1
                        contexts[pre_context][CHARS_KEY] = defaultdict(int)
                    contexts[pre_context][CHARS_KEY][cur_letter] += 1
            for key, value in contexts.items():
                vocab.update(value[CHARS_KEY].keys())

    # context is a string
    def prob(self, context, char):
        ## TODO 2.3
        probability = 0
        context_input = context
        for i in range(self.n+1):
            context = context_input[0-i:]
            # print("context {}".format(context))
            if i == 0:
                context = ''
            # print("sub context is [{}]".format(context))
            vocab = self.vocab[i]
            cur_prob = 0
            contexts = self.models[i]
            if self.k == 0:
                if context not in contexts:
                    # cur_prob = 1/(len(self.get_vocab()))
                    cur_prob = 1 / (len(vocab))
                else:
                    context_count = contexts[context][COUNT_KEY]
                    char_count = contexts[context][CHARS_KEY][char]
                    cur_prob = char_count / context_count
            else:
                if context not in contexts:
                    cur_prob = 1/(len(vocab))
                else:
                    v_count = len(vocab)
                    sum_n = 0
                    char_count = self.k
                    for key in contexts[context][CHARS_KEY]:
                        if key == char:
                            char_count += contexts[context][CHARS_KEY][key]
                        sum_n += contexts[context][CHARS_KEY][key]
                    cur_prob = char_count / (sum_n + self.k * v_count)
            # print("i is {} cur_prob is {}".format(i, cur_prob))
            probability += self.lambdas[i] * cur_prob

        return probability

    # def perplexity(self, text):
    #     # print("perplexity")
    #     ''' Returns the perplexity of text based on the n-grams learned by
    #         this model '''
    #     ## TODO 2.2 - see problem 2.2
    #     # return 0.56
    #     grams = ngrams(self.n, text)
    #     # print("grams is {}".format(grams))
    #     sum = 0

    #     for gram in grams:
    #         prob = self.prob(gram[0], gram[1])
    #         # print("prob is {}".format(prob))
    #         if prob == 0.0:
    #             continue
    #         log_new = math.log2(prob)
    #         sum += log_new
    #     avg = sum / len(grams)
    #     # print("avg is {} len of gram is {}".format(avg, len(grams)))
    #     return math.pow(2, 0 - avg)

In [23]:
# PennGrader - DO NOT CHANGE
interpolation_test_cases = [
    [(0, 1), ['abab'], ('a', 'a')],
    [(2, 1), ['abab', 'abcd'], ('~c', 'd')],
    [(2, 1), ['abeadab', 'abcd', 'eadb'], ('~a','b')],
    [(2, 1), ['abab', 'abcd', 'aaaaad'], ('bc', 'd')],
]

interpolation_test_results = []
for (n, k), corpus, (context, char) in interpolation_test_cases:
    nm_interpolation = NgramModelWithInterpolation(n, k)
    for text in corpus:
        nm_interpolation.update(text)
    interpolation_test_results.append(nm_interpolation.prob(context, char))

grader.grade(test_case_id = 'test_q23_interpolation', answer = interpolation_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [ ]:
# uninterpolated
n = 2
k = 0.1
filename1 = "shakespeare_sonnets.txt"
model_2_2_n2_k01 = create_ngram_model(NgramModel, 'shakespeare_input.txt', n, k)
with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = model_2_2_n2_k01.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "model_2_2_n2_k01", perplexity))


# interpolated perplexity
lambda_model_n1_k01 = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n, k)


with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = lambda_model_n1_k01.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "lambda_model_n1_k01", perplexity))


filaname1 shakespeare_sonnets.txt modelname model_2_2_n2_k01 perplexity 7.996946415762477
filaname1 shakespeare_sonnets.txt modelname lambda_model_n1_k01 perplexity 10.288650009339714


In [ ]:
# uninterpolated
n = 2
k = 0.5
filename1 = "shakespeare_sonnets.txt"
model_2_2_n2_k05 = create_ngram_model(NgramModel, 'shakespeare_input.txt', n, k)
with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = model_2_2_n2_k05.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "model_2_2_n2_k05", perplexity))


# interpolated perplexity
lambda_model_n1_k05 = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n, k)


with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = lambda_model_n1_k05.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "lambda_model_n1_k05", perplexity))


filaname1 shakespeare_sonnets.txt modelname model_2_2_n2_k05 perplexity 7.918440177460784
filaname1 shakespeare_sonnets.txt modelname lambda_model_n1_k05 perplexity 10.307018627007555


In [ ]:
# uninterpolated
n = 2
k = 1
filename1 = "shakespeare_sonnets.txt"
model_2_2_n2_k1 = create_ngram_model(NgramModel, 'shakespeare_input.txt', n, k)
with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = model_2_2_n2_k1.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "model_2_2_n2_k1", perplexity))


# interpolated perplexity
lambda_model_n1_k1 = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n, k)

with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = lambda_model_n1_k1.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "lambda_model_n1_k1", perplexity))


filaname1 shakespeare_sonnets.txt modelname model_2_2_n2_k1 perplexity 7.910100720888779
filaname1 shakespeare_sonnets.txt modelname lambda_model_n1_k1 perplexity 10.328866826678306


In [ ]:
# uninterpolated
n = 2
k = 2
filename1 = "shakespeare_sonnets.txt"
model_2_2_n2_k2 = create_ngram_model(NgramModel, 'shakespeare_input.txt', n, k)
with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = model_2_2_n2_k2.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "model_2_2_n2_k2", perplexity))


# interpolated perplexity
lambda_model_n1_k2 = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n, k)

with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = lambda_model_n1_k2.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "lambda_model_n1_k2", perplexity))


filaname1 shakespeare_sonnets.txt modelname model_2_2_n2_k2 perplexity 7.932029974368767
filaname1 shakespeare_sonnets.txt modelname lambda_model_n1_k2 perplexity 10.370481247382997


In [ ]:
# uninterpolated
n = 2
k = 3
filename1 = "shakespeare_sonnets.txt"
model_2_2_n2_k3 = create_ngram_model(NgramModel, 'shakespeare_input.txt', n, k)
with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = model_2_2_n2_k3.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "model_2_2_n2_k3", perplexity))


# interpolated perplexity
lambda_model_n1_k3 = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n, k)

with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = lambda_model_n1_k3.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "lambda_model_n1_k3", perplexity))

filaname1 shakespeare_sonnets.txt modelname model_2_2_n2_k3 perplexity 7.966470772488156
filaname1 shakespeare_sonnets.txt modelname lambda_model_n1_k3 perplexity 10.409329628452168


In [ ]:
# uninterpolated
n = 3
k = 0.1
lambdas = [0.25, 0.25, 0.25, 0.25]
filename1 = "shakespeare_sonnets.txt"

# interpolated perplexity
lambda_model_n1_k01_lambda1 = NgramModelWithInterpolation(n, k)
lambda_model_n1_k01_lambda1.set_lambdas(lambdas)

with open("shakespeare_input.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    lambda_model_n1_k01_lambda1.update(text)


with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = lambda_model_n1_k01_lambda1.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "lambda_model_n1_k01_lambda1", perplexity))

filaname1 shakespeare_sonnets.txt modelname lambda_model_n1_k01_lambda1 perplexity 7.961566982637742


In [ ]:
# uninterpolated
n = 3
k = 0.1
lambdas = [0.4, 0.3, 0.2, 0.1]
filename1 = "shakespeare_sonnets.txt"

# interpolated perplexity
lambda_model_n1_k01_lambda2 = NgramModelWithInterpolation(n, k)
lambda_model_n1_k01_lambda2.set_lambdas(lambdas)

with open("shakespeare_input.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    lambda_model_n1_k01_lambda2.update(text)


with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = lambda_model_n1_k01_lambda2.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "lambda_model_n1_k01_lambda2", perplexity))

filaname1 shakespeare_sonnets.txt modelname lambda_model_n1_k01_lambda2 perplexity 9.869056489356785


In [ ]:
# uninterpolated
n = 3
k = 0.1
lambdas = [0.1, 0.2, 0.3, 0.4]
filename1 = "shakespeare_sonnets.txt"

# interpolated perplexity
lambda_model_n1_k01_lambda3 = NgramModelWithInterpolation(n, k)
lambda_model_n1_k01_lambda3.set_lambdas(lambdas)

with open("shakespeare_input.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    lambda_model_n1_k01_lambda3.update(text)


with open(filename1, 'r', encoding='utf-8') as f:
    text = f.read()
    perplexity = lambda_model_n1_k01_lambda3.perplexity(text)
    print("filaname1 {} modelname {} perplexity {}".format(filename1, "lambda_model_n1_k01_lambda3", perplexity))

filaname1 shakespeare_sonnets.txt modelname lambda_model_n1_k01_lambda3 perplexity 6.836701359512738


- **Answer 2.3**: Experiment with a few different lambdas and values of k and discuss the effects of smoothing and interpolation in terms of perplexity. Alternatively, you can also try different number of n. You may still use text files from 2.2. [6 points]



In [ ]:
## Answer 2.3 in your PDF

# Section 3: Text Classification using N-Grams [20 points + 5 bonus]
**No PennGrader in this section, see `Deliverables` for grading details**
## Overview
Language models can be applied to text classification. If we want to classify a text $D$ into a category $c \in C={c_1, ..., c_N}$. We can pick the category $c$ that has the largest posterior probability given the text. That is,

$$ c^* = arg max_{c \in C} P(c|D) $$

Using Bayes rule, this can be rewritten as:

$$ c^* = arg max_{c \in C} P(D|c) P(c)$$

If we assume that all classes are equally likely, then we can just drop the $P(c)$ term:

$$ = arg max_{c \in C} P(D|c)$$

Here $P(D \mid c)$ is the likelihood of $D$ under category $c$, which can be computed by training language models for all texts associated with category $c$. This technique of text classification is drawn from [literature on authorship identification](http://www.aclweb.org/anthology/E/E03/E03-1053.pdf), where the approach is to learn a separate language model for each author, by training on a data set from that author. Then, to categorize a new text D, they use each language model to calculate the likelihood of D under that model, and pick the  category that assigns the highest probability to D.


## Try it!
We have provided you training and validation datsets consisting of the names of cities. The task is to predict the country a city is in. The following countries are including in the dataset.

```
af	Afghanistan
cn	China
de	Germany
fi	Finland
fr	France
in	India
ir	Iran
pk	Pakistan
za	South Africa
```

We'll set up a leaderboard for the text classification task. **Your job is to configure a set of language models that perform the best on the text classification task.** We will use the `city names` dataset, which you should have already downloaded. The test set has one unlabeled city name per line.

Your code should output a file `test_labels.txt` with one two-letter country code per line.

Feel free to extend the `NgramModel` or `NgramModelWithInterpolation` when creating your language model. Possible ideas to consider and experiment with when creating your model are utilizing a special end-of-text character, trying a new method for determining the vocab, and improving how your model handles novel characters.

## Deliverables
- **Answer 3.1:** Train your best model on the given training data (located in `train` folder), and report accuracy on the given validation data (located in `val` folder) for your final model. In order to receive full credit, your model must be able to outperform all the baselines. [5 points]

- **Answer 3.2:** Describe the parameters you used for the final submission, such as n , k and interpolation. Discuss how you made the decision and why your combination have better results, we encourage using a table to display all validation accuracies of your experiment process. Be sure to include a detailed error analysis: identify several examples of cities on which your final model is making errors, discuss potential reasons. [15 points]

- **Answer 3.3:** Make predictions on the given test data (`cities_test.txt`) and upload `test_labels.txt` to Gradescope, which contains one two-letter country code per line. Please note that testing accuracy should be valid and outperform the baseline in order to receive full credits (included in 3.1). Top 3 will be granted 5 bonus points.

In [24]:
COUNTRY_CODES = ['af', 'cn', 'de', 'fi', 'fr', 'in', 'ir', 'pk', 'za']

In [35]:
country_code_index_dict = dict()

for i in range(len(COUNTRY_CODES)):
    country_code_index_dict[COUNTRY_CODES[i]] = i
country_code_index_dict

{'af': 0,
 'cn': 1,
 'de': 2,
 'fi': 3,
 'fr': 4,
 'in': 5,
 'ir': 6,
 'pk': 7,
 'za': 8}

In [25]:
import zipfile

with zipfile.ZipFile('cities_train.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted/cities_train/')

In [26]:
import os

files = os.listdir('extracted/cities_train/train')
print(files)

['ir.txt', 'za.txt', 'in.txt', 'fi.txt', 'af.txt', 'cn.txt', 'de.txt', 'fr.txt', 'pk.txt']


In [27]:
import zipfile

with zipfile.ZipFile('cities_val.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted/cities_val/')

In [28]:
import os

files = os.listdir('extracted/cities_val/val')
print(files)

['ir.txt', 'za.txt', 'in.txt', 'fi.txt', 'af.txt', 'cn.txt', 'de.txt', 'fr.txt', 'pk.txt']


In [43]:
for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_train/train/{}.txt".format(country_code)
    with open(file_name, 'r', encoding="utf-8", errors="ignore") as f:
        cities = f.readlines()
        city_name_lengths = []
        for city in cities:
            city_name_lengths.append(len(city))
        city_name_lengths_arr = np.array(city_name_lengths)
        average = np.mean(city_name_lengths_arr)
        print("country {} city_name_avg_length {}".format(country_code, average))

country af city_name_avg_length 12.157666666666668
country cn city_name_avg_length 10.648666666666667
country de city_name_avg_length 13.557333333333334
country fi city_name_avg_length 11.443333333333333
country fr city_name_avg_length 12.631666666666666
country in city_name_avg_length 12.446666666666667
country ir city_name_avg_length 12.328666666666667
country pk city_name_avg_length 12.584666666666667
country za city_name_avg_length 12.169666666666666


In [44]:
for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_val/val/{}.txt".format(country_code)
    with open(file_name, 'r', encoding="utf-8", errors="ignore") as f:
        cities = f.readlines()
        city_name_lengths = []
        for city in cities:
            city_name_lengths.append(len(city))
        city_name_lengths_arr = np.array(city_name_lengths)
        average = np.mean(city_name_lengths_arr)
        print("country {} city_name_avg_length {}".format(country_code, average))

country af city_name_avg_length 12.25
country cn city_name_avg_length 10.38
country de city_name_avg_length 13.54
country fi city_name_avg_length 11.95
country fr city_name_avg_length 12.55
country in city_name_avg_length 12.01
country ir city_name_avg_length 12.19
country pk city_name_avg_length 11.92
country za city_name_avg_length 11.86


In [ ]:
n = 2
k = 1
country_to_models = dict()
for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_train/train/{}.txt".format(country_code)
    country_to_models[country_code] = create_ngram_model_lines(NgramModelWithInterpolation, file_name, n, k)

total_count = 0 # the number of cities
correct_count = 0
incorrect_count = 0

# for country_code in COUNTRY_CODES:
for country_code in ['cn']:
    file_name = "extracted/cities_val/val/{}.txt".format(country_code)
    with open(file_name, 'r', encoding='utf-8') as f:
        cities = f.readlines()

    for city in cities:
        total_count += 1
        min_perplexity = 1000.0
        min_country_code = 'af'
        for model_country_code in COUNTRY_CODES:
            nm_model = country_to_models[model_country_code]
            perplexity = nm_model.perplexity(city)
            # print("country_code: {} perplexity: {}".format(model_country_code, perplexity))
            if min_perplexity > perplexity:
                min_perplexity = perplexity
                min_country_code = model_country_code
        print("city is {} country {} predict_country {}".format(city, country_code, min_country_code))
        if min_country_code == country_code:
            correct_count += 1
        else:
            incorrect_count += 1

accuracy = correct_count / total_count
print("n is {} k is {} accuracy is {}".format(n, k, accuracy))

city is niuhuahsi
 country cn predict_country cn
city is mapingtai
 country cn predict_country cn
city is dashazhou
 country cn predict_country cn
city is shihchiayai
 country cn predict_country cn
city is dafo
 country cn predict_country af
city is diguai
 country cn predict_country cn
city is shaochialou
 country cn predict_country cn
city is tsaoyuan
 country cn predict_country cn
city is huolangzhangzi
 country cn predict_country cn
city is chayashan
 country cn predict_country af
city is qingquyao
 country cn predict_country cn
city is chaopeilo
 country cn predict_country cn
city is kuanshan
 country cn predict_country cn
city is kinong dzong
 country cn predict_country in
city is sifu
 country cn predict_country fi
city is haibei
 country cn predict_country cn
city is leiqiao
 country cn predict_country cn
city is huashan
 country cn predict_country cn
city is linangou
 country cn predict_country cn
city is xingmu gongshe
 country cn predict_country cn
city is minji
 country cn 

In [ ]:
n = 2
k = 1
country_to_models = dict()
for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_train/train/{}.txt".format(country_code)
    country_to_models[country_code] = create_ngram_model_lines(NgramModelWithInterpolation, file_name, n, k)

total_count = 0 # the number of cities
correct_count = 0
incorrect_count = 0

for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_val/val/{}.txt".format(country_code)
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()


    acc_bool = []
    i = 0
    for line in lines:
        #total_count += 1
        #min_perplexity = 100.0
        #min_country_code = 'af'
        model_perplexity = []
        for country_code in COUNTRY_CODES: # country_to_models.keys():
            nm_model = country_to_models[country_code]
            #perplexity = nm_model.perplexity(line)
            model_perplexity.append(nm_model.perplexity(line))

        model_scores = np.asarray(model_perplexity)
        #country prediction
        country_pred = COUNTRY_CODES[np.argmin(model_scores)]



        if i % 100 == 0:
            print('city = ', line)
            print('predicted country = ', country_pred)
            print('actual country = ', country_code)

            #if min_perplexity > perplexity:
            #    min_perplexity = perplexity
            #    min_country_code = country_code
        #if min_country_code == country_code:
        #    correct_count += 1
        #else:
        #    incorrect_count += 1
        acc_bool.append(country_pred == country_code)
        i =+ 1

#accuracy = correct_count / total_count
accuracy = sum(acc_bool) / len(acc_bool)
print("n is {} k is {} accuracy is {}".format(n, k, accuracy))

city =  sikhtopa

predicted country =  af
actual country =  za
city =  niuhuahsi

predicted country =  cn
actual country =  za
city =  stoudena

predicted country =  de
actual country =  za
city =  soppeenmaki

predicted country =  fi
actual country =  za
city =  torce

predicted country =  fr
actual country =  za
city =  sinceira grande

predicted country =  de
actual country =  za
city =  bonabdan

predicted country =  ir
actual country =  za
city =  bamochhi khurd

predicted country =  pk
actual country =  za
city =  az zawr

predicted country =  za
actual country =  za
n is 2 k is 1 accuracy is 0.61


In [ ]:
from zipfile import ZipFile
import io
import numpy as np
# new code
best_n = -1
best_k = -1
best_accuracy = 0
for n in range(1, 2):
    # for k in np.arange(0.1, 1, 0.2):
    for k in np.arange(1, 2):
        country_to_models = dict()
        for country_code in COUNTRY_CODES:
            file_name = "extracted/cities_train/train/{}.txt".format(country_code)
            country_to_models[country_code] = create_ngram_model_lines(NgramModelWithInterpolation, file_name, n, k)

        total_count = 0 # the number of cities
        correct_count = 0
        incorrect_count = 0

        for country_code in COUNTRY_CODES:
            file_name = "extracted/cities_val/val/{}.txt".format(country_code)
            with open(file_name, 'r', encoding='utf-8') as f:
                lines = f.readlines()

            for city in lines:
                total_count += 1
                min_perplexity = 100.0
                min_country_code = 'af'
                for model_country_code in COUNTRY_CODES:
                    nm_model = country_to_models[model_country_code]
                    perplexity = nm_model.perplexity(city)
                    if min_perplexity > perplexity:
                        min_perplexity = perplexity
                        min_country_code = country_code
                if min_country_code == country_code:
                    correct_count += 1
                else:
                    incorrect_count += 1
        accuracy = correct_count / total_count
        print("n is {} k is {} accuracy is {}".format(n, k, accuracy))
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_n = n
            best_k = k

print("best_n {} best_k {} best_accuracy {}".format(best_n, best_k, best_accuracy))

n is 1 k is 1 accuracy is 1.0
best_n 1 best_k 1 best_accuracy 1.0


In [ ]:
from zipfile import ZipFile
import io
import numpy as np
# new code
best_n = -1
best_k = -1
best_accuracy = 0
for n in range(1, 6):
    # for k in np.arange(0.1, 1, 0.2):
    for k in np.arange(0, 6):
        country_to_models = dict()
        for country_code in COUNTRY_CODES:
            file_name = "extracted/cities_train/train/{}.txt".format(country_code)
            country_to_models[country_code] = create_ngram_model_lines(NgramModelWithInterpolation, file_name, n, k)

        total_count = 0 # the number of cities
        correct_count = 0
        incorrect_count = 0

        for country_code in COUNTRY_CODES:
            file_name = "extracted/cities_val/val/{}.txt".format(country_code)
            with open(file_name, 'r', encoding='utf-8') as f:
                lines = f.readlines()

            for line in lines:
                total_count += 1
                min_perplexity = 100.0
                min_country_code = 'af'
                for model_country_code in country_to_models.keys():
                    nm_model = country_to_models[model_country_code]
                    perplexity = nm_model.perplexity(line)
                    if min_perplexity > perplexity:
                        min_perplexity = perplexity
                        min_country_code = model_country_code
                if min_country_code == country_code:
                    correct_count += 1
                else:
                    incorrect_count += 1
        accuracy = correct_count / total_count
        print("n is {} k is {} accuracy is {}".format(n, k, accuracy))
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_n = n
            best_k = k

print("best_n {} best_k {} best_accuracy {}".format(best_n, best_k, best_accuracy))

n is 1 k is 0 accuracy is 0.11222222222222222
n is 1 k is 1 accuracy is 0.6344444444444445
n is 1 k is 2 accuracy is 0.6322222222222222
n is 1 k is 3 accuracy is 0.6322222222222222
n is 1 k is 4 accuracy is 0.6344444444444445
n is 1 k is 5 accuracy is 0.63
n is 2 k is 0 accuracy is 0.11
n is 2 k is 1 accuracy is 0.6633333333333333
n is 2 k is 2 accuracy is 0.6533333333333333
n is 2 k is 3 accuracy is 0.65
n is 2 k is 4 accuracy is 0.6477777777777778
n is 2 k is 5 accuracy is 0.6488888888888888
n is 3 k is 0 accuracy is 0.08333333333333333
n is 3 k is 1 accuracy is 0.66
n is 3 k is 2 accuracy is 0.6555555555555556
n is 3 k is 3 accuracy is 0.6455555555555555
n is 3 k is 4 accuracy is 0.6466666666666666
n is 3 k is 5 accuracy is 0.6344444444444445
n is 4 k is 0 accuracy is 0.23333333333333334
n is 4 k is 1 accuracy is 0.6688888888888889
n is 4 k is 2 accuracy is 0.6588888888888889
n is 4 k is 3 accuracy is 0.6433333333333333
n is 4 k is 4 accuracy is 0.6322222222222222
n is 4 k is 5 accu

In [29]:
n = 4
k = 1
country_to_models_n4_k1 = dict()
for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_train/train/{}.txt".format(country_code)
    country_to_models_n4_k1[country_code] = create_ngram_model_lines(NgramModelWithInterpolation, file_name, n, k)


In [41]:
import numpy as np
from collections import defaultdict
country_incorrect_dict = defaultdict(int)
wrongly_country_country = defaultdict(int)


for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_val/val/{}.txt".format(country_code)
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    for city in lines:
        perplexities = []
        for model_country_code in COUNTRY_CODES:
            model = country_to_models_n4_k1[model_country_code]
            perplexities.append(model.perplexity(city))
        perplexities_np_arr = np.asarray(perplexities)
        country_pred = COUNTRY_CODES[np.argmin(perplexities_np_arr)]
        if country_code != country_pred:
            country_incorrect_dict[country_code] += 1
            wrongly_country_country[country_pred] += 1
            # actual_per = perplexities[country_code_index_dict[country_code]]
            # pred_per = perplexities[country_code_index_dict[country_pred]]
            # print("city name {} country_code {} actual_per {} predicted_country {} pred_per {}".format(city, country_code, actual_per, country_pred, pred_per))
            # print(perplexities)

print(country_incorrect_dict)
print(wrongly_country_country)

defaultdict(<class 'int'>, {'af': 38, 'cn': 5, 'de': 32, 'fi': 23, 'fr': 17, 'in': 57, 'ir': 51, 'pk': 34, 'za': 41})
defaultdict(<class 'int'>, {'fr': 32, 'fi': 24, 'cn': 50, 'ir': 34, 'pk': 36, 'za': 21, 'in': 26, 'af': 47, 'de': 28})


In [40]:
import numpy as np
from collections import defaultdict
country_incorrect_dict = defaultdict(int)


for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_val/val/{}.txt".format(country_code)
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    perplexities = []
    for city in lines:
        model = country_to_models_n4_k1[country_code]
        perplexities.append(model.perplexity(city))
    arr = np.array(perplexities)
    # Compute average
    average = np.mean(arr)
    print("country_code {} average perplexity {}".format(country_code, average))



#         for model_country_code in COUNTRY_CODES:
#             model = country_to_models_n4_k1[model_country_code]
#             perplexities.append(model.perplexity(city))
#         perplexities_np_arr = np.asarray(perplexities)
#         country_pred = COUNTRY_CODES[np.argmin(perplexities_np_arr)]
#         if country_code != country_pred:
#             country_incorrect_dict[country_code] += 1
#             # actual_per = perplexities[country_code_index_dict[country_code]]
#             # pred_per = perplexities[country_code_index_dict[country_pred]]
#             # print("city name {} country_code {} actual_per {} predicted_country {} pred_per {}".format(city, country_code, actual_per, country_pred, pred_per))
#             # print(perplexities)

# country_incorrect_dict

country_code af average perplexity 15.285629366307175
country_code cn average perplexity 11.25065682658001
country_code de average perplexity 16.173093892293153
country_code fi average perplexity 15.069182061183895
country_code fr average perplexity 14.223321236688978
country_code in average perplexity 16.028710295406537
country_code ir average perplexity 16.651319547846285
country_code pk average perplexity 13.895243750248396
country_code za average perplexity 16.366496181929197


In [ ]:
n = 4
k = 1
country_to_models = dict()
for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_train/train/{}.txt".format(country_code)
    country_to_models[country_code] = create_ngram_model_lines(NgramModelWithInterpolation, file_name, n, k)

results = []
with open('cities_test.txt', 'r', encoding='utf-8', errors="ignore") as f:
    lines = f.readlines()
    for city in lines:
        min_perplexity = 1000.0
        min_country_code = 'af'
        for country_code in COUNTRY_CODES:
            nm_model = country_to_models[country_code]
            perplexity = nm_model.perplexity(city)
            if min_perplexity > perplexity:
                min_perplexity = perplexity
                min_country_code = country_code
        results.append(min_country_code)

with open('label_text.txt', 'w', encoding='utf-8') as f:
    for item in results:
        f.write(item + '\n')
results

['fr',
 'pk',
 'fi',
 'af',
 'in',
 'fi',
 'za',
 'ir',
 'de',
 'ir',
 'fi',
 'fi',
 'af',
 'de',
 'fi',
 'in',
 'fi',
 'fr',
 'ir',
 'fi',
 'fi',
 'cn',
 'de',
 'ir',
 'in',
 'fi',
 'fr',
 'de',
 'de',
 'ir',
 'za',
 'fi',
 'cn',
 'ir',
 'fr',
 'pk',
 'fi',
 'fi',
 'de',
 'ir',
 'af',
 'za',
 'pk',
 'fi',
 'in',
 'ir',
 'de',
 'za',
 'cn',
 'cn',
 'fr',
 'za',
 'pk',
 'ir',
 'in',
 'fi',
 'af',
 'fr',
 'cn',
 'ir',
 'pk',
 'pk',
 'in',
 'cn',
 'fr',
 'af',
 'fi',
 'af',
 'cn',
 'cn',
 'in',
 'de',
 'cn',
 'pk',
 'cn',
 'de',
 'fi',
 'za',
 'ir',
 'cn',
 'in',
 'in',
 'af',
 'af',
 'de',
 'ir',
 'fr',
 'za',
 'ir',
 'in',
 'fi',
 'de',
 'fr',
 'cn',
 'fi',
 'cn',
 'za',
 'za',
 'fr',
 'pk',
 'fi',
 'cn',
 'za',
 'fr',
 'za',
 'fr',
 'ir',
 'fr',
 'fr',
 'af',
 'pk',
 'fr',
 'za',
 'cn',
 'pk',
 'pk',
 'fr',
 'fi',
 'ir',
 'fi',
 'cn',
 'de',
 'cn',
 'de',
 'cn',
 'pk',
 'pk',
 'pk',
 'cn',
 'de',
 'za',
 'fr',
 'in',
 'cn',
 'pk',
 'pk',
 'in',
 'de',
 'af',
 'de',
 'za',
 'in',
 'fi',

In [ ]:
len(results)

In [ ]:
# evaluate the perplexity  by using val.zip
with ZipFile('cities_val.zip') as zip_ref:
    # file_list = zip_ref.namelist()
    for country_code in COUNTRY_CODES:
        file_name = "val/{}.txt".format(country_code)
        nm_model = country_to_models[country_code]
        total_perplexity = 0
        count = 0
        with zip_ref.open(file_name) as binary_file:
            with io.TextIOWrapper(binary_file, encoding='utf-8', errors="ignore") as text_file:
                for line in text_file:
                    count += 1
                    # print("country_code {} line is {}".format(country_code, line))
                    total_perplexity += nm_model.perplexity(line)
                avg_perplexity = total_perplexity / count
                print("avg_perplexity {}".format(avg_perplexity))


avg_perplexity 12.003981204110119
avg_perplexity 8.407378283254715
avg_perplexity 12.489677179197544
avg_perplexity 11.817241785735549
avg_perplexity 11.133402849281921
avg_perplexity 11.80525228174665
avg_perplexity 12.952414719821077
avg_perplexity 10.197368535137043
avg_perplexity 13.052958244506758


In [54]:
lambdas = [1/15, 2/15, 3/15, 4/15, 5/15]
country_to_models_lambda = dict()

# train the models
for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_train/train/{}.txt".format(country_code)
    country_to_models_lambda[country_code] = NgramModelWithInterpolation(4, 0.5)
    country_to_models_lambda[country_code].lambdas = lambdas
    model = country_to_models_lambda[country_code]
    with open(file_name, encoding='utf-8', errors='ignore') as f:
        for line in f:
            model.update(line.strip())



In [55]:
country_to_models_lambda["af"].models

[{'': {'count': 33472,
   'chars': defaultdict(int,
               {'g': 709,
                'a': 6812,
                'r': 1722,
                'v': 120,
                's': 1320,
                'h': 2523,
                'k': 1793,
                'l': 1628,
                'i': 1544,
                'b': 744,
                'n': 1439,
                ' ': 1858,
                'e': 1687,
                'y': 979,
                'd': 1062,
                '-': 703,
                'u': 981,
                'j': 273,
                'm': 952,
                'o': 952,
                'f': 764,
                't': 699,
                'z': 452,
                'c': 332,
                '`': 228,
                'q': 403,
                'w': 393,
                "'": 140,
                'p': 252,
                '8': 1,
                'x': 3,
                '(': 1,
                '1': 1,
                ')': 1,
                '"': 1})}},
 {'~': {'count': 3000,
   'chars': 

In [59]:
total_count = 0
correct_count = 0
incorrect_count = 0
for country_code in COUNTRY_CODES:
    file_name = "extracted/cities_val/val/{}.txt".format(country_code)
    with open(file_name, 'r', encoding='utf-8', errors="ignore") as f:
        lines = f.readlines()

    for city in lines:
        total_count += 1
        per_scores = []
        for model_country_code in COUNTRY_CODES:
            nm_model = country_to_models_lambda[model_country_code]
            perplexity = nm_model.perplexity(city)
            per_scores.append(perplexity)
        min_country_code = COUNTRY_CODES[np.argmin(per_scores)]
        if min_country_code == country_code:
            correct_count += 1
        else:
            incorrect_count += 1

accuracy = correct_count / total_count

accuracy

0.6766666666666666

In [61]:
# use model with lambda
results = []
with open('cities_test.txt', 'r', encoding='utf-8', errors="ignore") as f:
    lines = f.readlines()
    for city in lines:
        per_scores = []
        for country_code in COUNTRY_CODES:
            nm_model = country_to_models_lambda[country_code]
            per_scores.append(nm_model.perplexity(city))
        per_scores_arr = np.asarray(per_scores)
        min_country_code = COUNTRY_CODES[np.argmin(per_scores_arr)]
        results.append(min_country_code)

results

['pk',
 'fi',
 'fi',
 'af',
 'in',
 'za',
 'fi',
 'ir',
 'de',
 'ir',
 'in',
 'fi',
 'af',
 'fr',
 'fi',
 'za',
 'fi',
 'fr',
 'ir',
 'fi',
 'fi',
 'cn',
 'pk',
 'ir',
 'za',
 'fi',
 'de',
 'de',
 'de',
 'ir',
 'pk',
 'fi',
 'cn',
 'af',
 'fr',
 'cn',
 'de',
 'fi',
 'de',
 'ir',
 'af',
 'za',
 'za',
 'in',
 'in',
 'ir',
 'de',
 'za',
 'cn',
 'cn',
 'fr',
 'za',
 'ir',
 'ir',
 'in',
 'fi',
 'in',
 'fr',
 'cn',
 'cn',
 'pk',
 'pk',
 'in',
 'cn',
 'fr',
 'af',
 'fi',
 'af',
 'cn',
 'cn',
 'in',
 'de',
 'cn',
 'ir',
 'cn',
 'de',
 'in',
 'za',
 'ir',
 'ir',
 'in',
 'in',
 'af',
 'af',
 'de',
 'ir',
 'fr',
 'za',
 'pk',
 'fr',
 'fi',
 'de',
 'fr',
 'cn',
 'ir',
 'cn',
 'za',
 'de',
 'fr',
 'pk',
 'fi',
 'cn',
 'za',
 'fr',
 'za',
 'fr',
 'ir',
 'fr',
 'fr',
 'af',
 'cn',
 'fr',
 'za',
 'cn',
 'pk',
 'in',
 'fr',
 'fi',
 'cn',
 'fi',
 'cn',
 'de',
 'cn',
 'fr',
 'cn',
 'in',
 'pk',
 'pk',
 'cn',
 'de',
 'za',
 'fr',
 'in',
 'cn',
 'pk',
 'pk',
 'in',
 'de',
 'af',
 'de',
 'za',
 'in',
 'fi',

In [ ]:
## Answer 3.1 and 3.2 in your PDF ###

# Submission
### Congratulation on finishing your homework! Here are the deliverables you need to submit to GradeScope
- This notebook and py file: rename to `homework3.ipynb` and `homework3.py`. You can download the notebook and py file by going to the top-left corner of this webpage, `File -> Download -> Download .ipynb/.py`
- Your `report.pdf`
  - including answers to 1.5, 2.2, 2.3, 3.1, and 3.2
- `test_labels.txt` from `Section 3`